In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

import random
import math

from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

import features as feat

df = pd.read_csv("train.csv")

#Create df in place with all the needed features and prepared values.
def prepare_features(df):
    df['SexuponOutcome'].fillna('Unknown', inplace=True)
    df['AgeuponOutcome'].fillna("-1 months", inplace=True)
    
    df['hasName'] = df['Name'].apply(lambda x: feat.hasName(x))
    df['intact'] = df['SexuponOutcome'].apply(lambda x: feat.intact(x))
    df['gender'] = df['SexuponOutcome'].apply(lambda x: feat.gender(x))
    df['age'] = df['AgeuponOutcome'].apply(lambda x: feat.convertAgeToMonths(x))
    df['isShihTzu'] = df['Breed'].apply(lambda x: feat.isShihTzu(x))
    df['isAggressive'] = df['Breed'].apply(lambda x: feat.isAggressive(x))
    df['isMix'] = df['Breed'].apply(lambda x: feat.isMix(x))
    df['isMultiColour'] = df['Color'].apply(lambda x: feat.isMultiColour(x))

    df['isToy'] = df['Breed'].apply(lambda x: feat.isToy(x))
    df['isWorking'] = df['Breed'].apply(lambda x: feat.isWorking(x))
    df['isSporting'] = df['Breed'].apply(lambda x: feat.isSporting(x))
    df['isHound'] = df['Breed'].apply(lambda x: feat.isHound(x))
    df['isNonSporting'] = df['Breed'].apply(lambda x: feat.isNonSporting(x))
    df['isHerding'] = df['Breed'].apply(lambda x: feat.isHerding(x))
    df['isTerrier'] = df['Breed'].apply(lambda x: feat.isTerrier(x))
    df['isShortHair'] = df['Breed'].apply(lambda x: feat.isShortHair(x))
    df['isMediumHair'] = df['Breed'].apply(lambda x: feat.isMediumHair(x))
    df['isLongHair'] = df['Breed'].apply(lambda x: feat.isLongHair(x))
    df['isTabby'] = df['Color'].apply(lambda x: feat.isTabby(x))
    
    df['AnimalType'] = df['AnimalType'].apply(lambda x: feat.convertAnimalType(x))
    
    df.drop(['AnimalID', 
             'DateTime', 
             'OutcomeSubtype',
             'SexuponOutcome',
             'AgeuponOutcome',
             'Breed',
             'Color',
             'DateTime',
             'Name'], axis=1, inplace=True)
    
prepare_features(df)

# Selecting 40% of Training set for actual training.
df = df.iloc[np.random.permutation(len(df))]
df.reset_index(inplace=True, drop=True)
split = int(len(df) * .4)

xTrain = df.iloc[:split + 1,1:] #iloc == [), loc == [] for ranges.
yTrain = df.loc[:split,'OutcomeType']

xTest = df.iloc[split:,1:]
yTest = df.loc[split:,'OutcomeType']

# Train classifer.
xTrain = xTrain.as_matrix()
yTrain = np.array(yTrain)
nodes = (len(xTrain[0]) + len(np.unique(yTrain))) / 2

clf = RandomForestClassifier(n_estimators = 100,
                            max_features = int(math.sqrt(len(xTrain[0]))),
                            oob_score = True,
                            )

rfc = clf.fit(xTrain,yTrain)
rfc.score(xTest, yTest)
    


0.59807956104252402